In [54]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

In [55]:
df = pd.read_excel('C:/Users/ADMIN/Desktop/python/python/python/customer_segment/Online Retail.xlsx',sheet_name='Online Retail') 

In [56]:
df.shape

(541909, 8)

In [57]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [58]:
# data cleaning
# 1. Loai bo cac don dat hang da huy
df = df.loc[df['Quantity'] > 0] 
df.shape


(531285, 8)

In [59]:
# 2. Loai bo ho so khong co Customer ID
df = df[pd.notnull(df['CustomerID'])]
df.shape

(397924, 8)

In [60]:
# 3. Bo di thang khong du ngay
df['InvoiceDate'].max()


Timestamp('2011-12-09 12:50:00')

In [61]:
df = df.loc[df['InvoiceDate']<'2011-12-01']
df.shape

(380620, 8)

In [62]:
# Tinh tong doanh thu (Quantity * UnitPrice)
df['Sales'] = df['Quantity'] * df['UnitPrice']
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [63]:
# Group nhom khach hang
customer_df = df.groupby('CustomerID').agg({
    'Sales': sum,
    'InvoiceNo': lambda x: x.nunique()
})
customer_df.columns = ['TotalSales','OrderCount']
customer_df['AvgOrderValue'] = customer_df['TotalSales']/customer_df['OrderCount']

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_8364\3994054685.py:2: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  customer_df = df.groupby('CustomerID').agg({


In [64]:
customer_df.head()

,TotalSales,OrderCount,AvgOrderValue
CustomerID,,,
12346.0,77183.60,1,77183.600000
12347.0,4085.18,6,680.863333
12348.0,1797.24,4,449.310000
12349.0,1757.55,1,1757.550000
12350.0,334.40,1,334.400000


In [65]:
# xep hang du lieu (dung method = 'first')
# khi gap trung lap thu hang se duoc gan theo thu tu xuat hien.
# Gia tri nao xuat hien truoc se nhan thu hang thap hon
rank_df = customer_df.rank(method='first')
rank_df.head()

,TotalSales,OrderCount,AvgOrderValue
CustomerID,,,
12346.0,4290.0,1.0,4298.0
12347.0,3958.0,3470.0,3888.0
12348.0,3350.0,2861.0,3303.0
12349.0,3321.0,2.0,4238.0
12350.0,1241.0,3.0,2561.0


In [66]:
# chuan hoa du lieu, do lech chuan la 1
normalized_df = (rank_df - rank_df.mean())/rank_df.std()
normalized_df.head()

,TotalSales,OrderCount,AvgOrderValue
CustomerID,,,
12346.0,1.724999,-1.731446,1.731446
12347.0,1.457445,1.064173,1.401033
12348.0,0.967466,0.573388,0.929590
12349.0,0.944096,-1.730641,1.683093
12350.0,-0.732148,-1.729835,0.331622


In [67]:
normalized_df.describe()

,TotalSales,OrderCount,AvgOrderValue
count,4298.000000,4298.000000,4.298000e+03
mean,0.000000,0.000000,-6.612776e-18
std,1.000000,1.000000,1.000000e+00
min,-1.731446,-1.731446,-1.731446e+00
25%,-0.865723,-0.865723,-8.657232e-01
50%,0.000000,0.000000,0.000000e+00
75%,0.865723,0.865723,8.657232e-01
max,1.731446,1.731446,1.731446e+00


In [68]:
# phan tich phan cum, dung thuat toan K-mean
# xay dung va dieu chinh mo hinh
# chia du lieu thanh 4 phan doan
kmeans = KMeans(n_clusters=4).fit(normalized_df[['TotalSales','OrderCount','AvgOrderValue']])
kmeans


KMeans(n_clusters=4)

In [74]:
# truy xuat du lieu
kmeans.labels_


array([2, 1, 1, ..., 3, 0, 1])

In [ ]:
kmeans.cluster_centers_

In [75]:
fous_cluster_df = normalized_df[['TotalSales','OrderCount','AvgOrderValue']].copy(deep=True)
fous_cluster_df['Cluster'] = kmeans.labels_

In [76]:
fous_cluster_df.head()

,TotalSales,OrderCount,AvgOrderValue,Cluster
CustomerID,,,,
12346.0,1.724999,-1.731446,1.731446,2
12347.0,1.457445,1.064173,1.401033,1
12348.0,0.967466,0.573388,0.929590,1
12349.0,0.944096,-1.730641,1.683093,2
12350.0,-0.732148,-1.729835,0.331622,2
